In [ ]:
from google.colab import drive
from os.path import join  

def mount_drive(ROOT):
    drive.mount(ROOT, force_remount=True)

ROOT = '/content/drive'
mount_drive(ROOT)

MY_GOOGLE_DRIVE_PATH = 'MyDrive/RL_NLP/' 
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)   
%cd "{PROJECT_PATH}"

In [ ]:
!pip3 install virtualenv
!virtualenv -p python3.8.10 python38

!source /content/drive/MyDrive/Colab/RL_NLP/python38/bin/activate
!python --version

In [ ]:
!pip install -r "{PROJECT_PATH}/metadata/requirements.txt"

## First time only run.

!git clone https://github.com/Farama-Foundation/gym-examples

%cd gym-examples

!pip install -e .

%cd ..

# RUN FROM HERE.

In [ ]:
import numpy as np 
import pandas as pd 
import time
import utils as U
import model as M
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt 
import json
import copy

In [ ]:
!pip install -e .

In [ ]:
import gym
%cd gym-examples
import gym_examples
%cd ..
import numpy as np

episodes=2
env = gym.make('gym_examples/RlNlpWorld-v0',render_mode="rgb_array")

### Random Policy.

In [ ]:
def policy(observation):
    return np.random.randint(0,6)

dbg=True
for _ in range(episodes):
    cumulative_reward,steps=0,0
    observation, info = env.reset(seed=42)
    cnt=0
    while True:
        action = policy(observation)  # User-defined policy function
        observation, reward, terminated, truncated, info = env.step(action)
        cumulative_reward+=reward
        steps+=1
        time.sleep(0.5)
        if dbg==True:
            print(f'cumulative_reward {cumulative_reward}; action {action}')
            print(observation["visual"].shape)
            output=U.crop_resize(observation["visual"])
            print(output.shape)
            plt.imshow(output)
            plt.savefig('SuperReduced.png')
            output_=np.mean(output,axis=2)
            plt.imshow(output_)
            plt.savefig('BW.png')
            print(output_.shape)
            break
        if terminated or truncated:
            break
    print(f'Cumulative Reward ~ {cumulative_reward}; TimeTaken ~ {steps}')
env.close()

### Q-Learning w/o NLP. (NAIVE MODEL)

In [ ]:
num_actions=6
update_after_actions=2**8
update_target_network=2**13
frame_cnt,epsilon_greedy=0,100
episodes,mxSteps=2,50
episode_reward_hist,episode_avg_reward=[],[]
epsilon,epsilon_min,epsilon_max=1.0,0.005,1.0 
epsilon_interval = (
    epsilon_max - epsilon_min
) 
epsilon_greedy_frames = 100000.0
batch_size=2**10
action=-1
replay_mem={'state':[],'next_state':[],'action':[],'reward':[],'small_n_state':[]}
mx_replay_memory=100000
gamma=0.9

# MODEL
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=M.NNModel().to(device)
model_traget=M.NNModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)
epochs=20

for _ in range(episodes):
    episode_reward=0
    state, info = env.reset(seed=42)
    for __ in range(mxSteps):
        frame_cnt+=1
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)
        if frame_cnt<=epsilon_greedy or epsilon>np.random.rand(1)[0]:
            action=np.random.choice(num_actions)
            # with open('my_dict.json', 'w') as f:
            #     U.phi(state)
            #     state["visual"]=state["visual"].tolist()
            #     json.dump(state,f)
        else:
            action=(M.predict(model,[state],device))[1]
        next_state,reward,terminated,truncated,info=env.step(action)
        episode_reward+=reward 
        temp_next_state=copy.deepcopy(next_state) #To Save Small Image.
        U.phi(state);replay_mem['state'].append(state)
        U.phi(next_state);replay_mem['next_state'].append(next_state)
        U.phiXtra(temp_next_state);replay_mem['small_n_state'].append(temp_next_state)
        replay_mem['action'].append(U.oneHot(num_actions,action))
        replay_mem['reward'].append(reward)
        state=next_state 
        if True or frame_cnt%update_after_actions==0 and len(replay_mem['reward'])>batch_size:
            choices=np.random.choice( range( len(replay_mem['reward']) ), size=batch_size )
            STATE=[replay_mem['state'][id] for id in choices]
            NEXT_STATE=[replay_mem['next_state'][id] for id in choices]
            SMALL_NEXT_STATE=[replay_mem['small_n_state'][id] for id in choices]
            ACTION=np.array([replay_mem['action'][id] for id in choices])
            REWARD=np.array([replay_mem['reward'][id] for id in choices])
            IMG=np.array([_state_["visual"] for _state_ in STATE])
            reward_true=REWARD+gamma*(M.predict(model_traget,NEXT_STATE,device))[0]
            continue
            for epoch in range(epochs):
                M.train(model,reward_true,STATE,SMALL_NEXT_STATE,ACTION,device,optimizer,epoch)
        if frame_cnt%update_target_network==0:
            model_traget.load_state_dict(model.state_dict())
        if len(replay_mem['reward'])>mx_replay_memory:
            del replay_mem['state'][:1]
            del replay_mem['next_state'][:1]
            del replay_mem['action'][:1]
            del replay_mem['reward'][:1]
        if terminated or truncated:
            break
    
    episode_reward_hist.append(episode_reward)
    episode_avg_reward.append(episode_reward/__)

# Save Relevant Results
np.save('results/ep_tot.npy',np.array(episode_reward_hist))
np.save('results/ep_avg.npy',np.array(episode_avg_reward))
torch.save(model.state_dict(),'results/model.ml')

In [ ]:
# Loading Results.
episode_reward_hist=np.load('results/ep_tot.npy')
episode_avg_reward=np.load('results/ep_avg.npy')
U.plot(episode_reward_hist,'Total Reward','Total Reward V/S Episodes')
U.plot(episode_avg_reward,'Avg. Reward','Avg. Reward Per Step V/S Episodes')
model_tr=M.NNModel().to(device)
model_tr.load_state_dict(torch.load('results/model.ml'))

In [ ]:
print(replay_mem)

In [ ]:
len(replay_mem["state"])

In [ ]:
len(replay_mem["action"])

### TBD

~~1. image training reduce.~~

~~2. save model_weights in File.~~

~~3. save replay memory in File.~~[Will do if necessary]

4. check whether for all weights -> requries_grad=True
5. upload the code in google collab and make it train.
6. write instruction auto gen code. 